In [7]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa, librosa.display

In [8]:
PATH_METADATA = "musicnet_metadata.csv"

PATH_DATA = "musicnet/data/"
PATH_LABELS = "musicnet/labels/"

PATH_TRAIN_DATA = "musicnet/train_data/"
PATH_TRAIN_LABELS = "musicnet/train_labels/"
PATH_TEST_DATA = "musicnet/test_data/"
PATH_TEST_LABELS = "musicnet/test_labels/"

In [9]:
ORIGINAL_SR = 44100
TARGET_SR = 16000
FMIN = librosa.note_to_hz("A0")
FMIN_MIDI_INDEX = librosa.note_to_midi("A0")
N_NOTES = 88
BINS_PER_NOTE = 1
BINS_PER_OCTAVE = 12 * BINS_PER_NOTE
N_BINS = N_NOTES * BINS_PER_NOTE

WINDOW_LENGTH = 2048
HOP_LENGTH = 512

frac_sr = TARGET_SR / ORIGINAL_SR
sample_indexer = frac_sr / HOP_LENGTH

In [10]:
import mido
from mido import Message, MidiFile, MidiTrack

def data_to_midi(filename, data, bpm):

    FRAME_DURATION = HOP_LENGTH/TARGET_SR # Frame duration in seconds
    PPQN = (1/FRAME_DURATION)*16 # Pulses per quarter note
    TEMPO = mido.bpm2tempo(bpm) # MIDI Tempo, microseconds per beat
    
    n_notes, n_frames = data.shape

    # pad zeros so we can acknowledge inital and ending events
    data = np.pad(data, [(0, 0), (1, 1)], 'constant')

    mid = MidiFile()
    
    for i in range(n_notes):
        note = i + FMIN_MIDI_INDEX # add the MIDI index
        changes = np.nonzero(np.diff(data[i]))[0] # list with the index where the array changes from 0 to 1 or vice versa
        if len(changes):
            t = round(mido.second2tick(FRAME_DURATION*changes[0], PPQN, TEMPO))
            track = MidiTrack()
            track.append(Message('program_change', program=0, time=t)) # program 0 is Acoustic Grand Piano
            for j in np.diff(changes):
                t = round(mido.second2tick(FRAME_DURATION*j, PPQN, TEMPO))
                track.append(Message('note_on', channel=0, note=note, velocity=64, time=t))
                track.append(Message('note_off', channel=0, note=note, velocity=0, time=0))
            
            mid.tracks.append(track)
    
    mid.save(filename+'.mid')

In [12]:
filename = "1759"
data = np.load(filename+"_prediction.npy")

bpm = 106 # Beats per minute
data_to_midi(filename, data, bpm)